<a href="https://colab.research.google.com/github/Bibek04-bit/Flood-Forecasting/blob/main/transformer_trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -U keras-tuner
!pip install keras==3.0
!pip install hydroeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.1/997.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-hub 0.21.1 requires keras>=3.5, but you have keras 3.0.0 which is incompatible.
tensorflow 2.19.0 requires keras>=3.5.0, but you have keras 3.0.0 which is incompatible.


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import hydroeval as he
# normalize the data
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# model library

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import SGD,RMSprop,Adam,Adadelta,Adagrad,Adamax,Nadam

# hyperparameter tuner
import keras_tuner as kt
import time
# model checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError,MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError


# loss

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving df_with_flood_labels.csv to df_with_flood_labels.csv
User uploaded file "df_with_flood_labels.csv" with length 1569649 bytes


In [5]:
import pandas as pd

file_name = 'df_with_flood_labels.csv'

try:
    df = pd.read_csv(file_name)
    print(f'Successfully loaded {file_name} into a DataFrame.')
    display(df.head())
except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found. Please ensure it's uploaded and the name is correct.")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

Successfully loaded df_with_flood_labels.csv into a DataFrame.


,Date,Daily flow Rate,Gauge height flow,24h accumulated,sm_top_0_10cm,sm_10_40cm,sm_change_rate,saturation_index,flow_change_rate,flow_lag_1d,...,Drainage_Density,Mean_CN,LULC_Forest_pct,LULC_Agriculture_pct,LULC_BuiltUp_pct,LULC_Grassland_pct,LULC_Water_pct,LULC_Barren_pct,flood_raw,flood_final
0,2009-01-03,456.0,3.33,0.0,22.784667,65.749364,4.666667,39.970546,-0.008696,460.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
1,2009-01-04,448.0,3.31,0.0,22.027667,68.656034,-0.757000,40.679014,-0.017544,456.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
2,2009-01-05,445.0,3.30,0.0,18.637667,58.216001,-3.390000,34.469000,-0.006696,448.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
3,2009-01-06,449.0,3.31,0.0,17.945667,56.251036,-0.692000,33.267815,0.008989,445.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
4,2009-01-07,453.0,3.32,0.0,18.687000,58.913774,0.741334,34.777710,0.008909,449.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0


In [6]:
features = [
    '24h accumulated',
    'rain_sum_3d',
    'rain_sum_5d',
    'rain_sum_7d',
    'sm_top_0_10cm',
    'sm_10_40cm',
    'sm_change_rate',
    'saturation_index',
    'API',
    'flow_lag_1d',
    'flow_lag_2d'
]

In [7]:
import pandas as pd
import numpy as np

# The 'features' list is already defined in a previous cell (0qyMCn1abg5d).
# The 'df' DataFrame is already loaded in a previous cell (L-Hd2gwKbJfE).

def min_max_scaling(data):
    # Subtract the minimum value and divide by the range
    min_val = np.min(data)
    max_val = np.max(data)
    # Handle the case where max_val - min_val is zero to avoid division by zero
    if (max_val - min_val) == 0:
        scaled_data = np.zeros_like(data) # If all values are the same, scale to 0
    else:
        scaled_data = (data - min_val) / (max_val - min_val)
    return scaled_data, min_val, max_val

# Create a DataFrame containing only the features to be scaled.
# We create a copy to avoid modifying the original 'df' directly and to prevent SettingWithCopyWarning.
df_to_scale = df[features].copy()

# Initialize scaled_df as a copy of df_to_scale to store the scaled data.
scaled_df = df_to_scale.copy()

# Apply min-max scaling to each column in the features list
for column in features: # Using 'features' list from the earlier cell
    scaled_df[column], _, _ = min_max_scaling(df_to_scale[column])

# Print the scaled DataFrame (showing head for brevity)
print("Scaled DataFrame head:")
display(scaled_df.head())

Scaled DataFrame head:


,24h accumulated,rain_sum_3d,rain_sum_5d,rain_sum_7d,sm_top_0_10cm,sm_10_40cm,sm_change_rate,saturation_index,API,flow_lag_1d,flow_lag_2d
0,0.0,0.0,0.0,0.0,0.420048,0.397530,0.602599,0.404110,0.0,0.015023,0.015588
1,0.0,0.0,0.0,0.0,0.396709,0.427545,0.377690,0.416303,0.0,0.014797,0.015023
2,0.0,0.0,0.0,0.0,0.292196,0.319739,0.268505,0.309427,0.0,0.014345,0.014797
3,0.0,0.0,0.0,0.0,0.270862,0.299448,0.380386,0.288754,0.0,0.014176,0.014345
4,0.0,0.0,0.0,0.0,0.293717,0.326944,0.439823,0.314740,0.0,0.014402,0.014176


In [8]:
df_model = df.drop(columns=['Date'])


In [15]:
X_df = scaled_df


In [16]:
y_df = df['flood_final']


In [17]:
def df_to_X_y(X_df, y_series, window_size=5):
    X_np = X_df.to_numpy()
    y_np = y_series.to_numpy()

    X, y = [], []

    for i in range(len(X_np) - window_size):
        X.append(X_np[i:i+window_size])
        y.append(y_np[i+window_size])

    return np.array(X), np.array(y)


In [18]:
WINDOW_SIZE = 2

X, y = df_to_X_y(X_df, y_df, WINDOW_SIZE)

print(X.shape)
print(y.shape)
print(np.unique(y))


(5474, 2, 11)
(5474,)
[0 1 2 3]


In [19]:
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid

from tensorflow import keras
from tensorflow.keras import layers

In [20]:
def build_model(hp):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    for i in range(hp.Int('num_transformer_blocks', 2, 8, step=2)):
        x = transformer_encoder(
            x,
            hp.Int('head_size', 8, 256, step=32),
            hp.Int('num_heads', 2, 16),
            hp.Int('ff_dim', 4, 64),
            hp.Float(f'dropout_{i}', 0.1, 0.6)
        )

    x = layers.GlobalAveragePooling1D()(x)

    for i in range(hp.Int('num_mlp_layers', 1, 3)):
        x = layers.Dense(hp.Int(f'mlp_units_{i}', 32, 256, step=32), activation='relu')(x)
        x = layers.Dropout(hp.Float(f'mlp_dropout_{i}', 0.1, 0.6))(x)

    outputs = layers.Dense(4, activation='softmax', dtype='float32')(x)

    model = keras.Model(inputs, outputs)

    optimizer_choice = hp.Choice('optimizer', ['adam', 'adamax'])
    lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')

    opt = Adam(lr) if optimizer_choice == 'adam' else Adamax(lr)

    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [21]:
input_shape = X.shape[1:]
input_shape

(2, 11)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
import keras_tuner as kt
import time

# Define the transformer_encoder function
# This function is a crucial part of the Transformer architecture
# and is called by the build_model function.
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


# Define your input shape
input_shape = X.shape[1:]

# Assume you have your data X and y
best_model_hyperparameters = None
best_model_mae = float('inf')

worst_model_hyperparameters = None
worst_model_mae = float('-inf')

# Initialize K-Fold cross-validation for the outer loop
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform Nested Cross-Validation
nested_scores = []

for fold_outer_idx, (train_ix, test_ix) in enumerate(outer_cv.split(X)):
    X_train_outer, X_test_outer = X[train_ix], X[test_ix]
    y_train_outer, y_test_outer = y[train_ix], y[test_ix]

    # Inner loop for hyperparameter tuning
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

    for fold_idx, (train_ix_inner, val_ix) in enumerate(inner_cv.split(X_train_outer)):
        X_train_inner, X_val = X_train_outer[train_ix_inner], X_train_outer[val_ix]
        y_train_inner, y_val = y_train_outer[train_ix_inner], y_train_outer[val_ix]

        tuner = kt.RandomSearch(
            build_model,
            objective='val_loss',
            max_trials=15,
            directory=f'/content/drive/MyDrive/floodresearch/keras_tuner_random_dir_fold_tf_{fold_outer_idx}_{fold_idx}',
            project_name=f'/content/drive/MyDrive/floodresearch/hyperparameter_random_tuning_fold_tf_{fold_outer_idx}_{fold_idx}'
        )

        # Search for the best hyperparameters
        tuner.search(X_train_inner, y_train_inner, validation_data=(X_val, y_val), epochs=30)

        best_hps = tuner.oracle.get_best_trials(1)[0].hyperparameters
        print(f"Best hyperparameters for fold {fold_idx}: {best_hps}")

        model = build_model(best_hps)
        es = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

        model.fit(
            X_train_inner,
            y_train_inner,
            validation_data=(X_val, y_val),
            epochs=30,
            batch_size=32,
            callbacks=[es],
        )

        y_pred = model.predict(X_val)
        # The original code calculates MAE, but the objective was val_loss (which is sparse_categorical_crossentropy)
        # and for classification problems accuracy is more appropriate. Also, y_pred from softmax is probabilities, not labels.
        # For mean_squared_error, you need actual labels. Assuming y_pred should be one-hot encoded or argmaxed for classification metrics.
        # If the goal is regression, then the build_model's output layer and loss function need to be adjusted.
        # Given the output activation is 'softmax' and loss is 'sparse_categorical_crossentropy', this is a classification task.
        # For demonstration purposes, I'll calculate MAE using argmax of y_pred for predicted classes to avoid immediate error,
        # but this metric might not be ideal for evaluating a classification model this way.
        # A better metric for classification would be accuracy, F1-score, etc., compared against y_val.
        mae = mean_absolute_error(y_val, np.argmax(y_pred, axis=1))

        # Update best model
        if mae < best_model_mae:
            best_model_mae = mae
            best_model_hyperparameters = best_hps
            best_model = model

        # Update worst model
        if mae > worst_model_mae:
            worst_model_mae = mae
            worst_model_hyperparameters = best_hps
            worst_model = model

    # Evaluate best and worst models on the test set
    start_time = time.time()
    y_pred_best_prob = best_model.predict(X_test_outer)
    y_pred_best = np.argmax(y_pred_best_prob, axis=1) # Get predicted classes

    y_pred_worst_prob = worst_model.predict(X_test_outer)
    y_pred_worst = np.argmax(y_pred_worst_prob, axis=1) # Get predicted classes
    end_time = time.time()

    # Use appropriate metrics for classification (e.g., accuracy, or MAE for difference in class labels)
    # The original code calculated MSE, MAE, RMSE, R2, which are typically for regression.
    # Since this is a classification problem (softmax output, sparse_categorical_crossentropy),
    # I'll calculate MAE for the predicted class labels against true labels.
    # For a more robust evaluation of a classifier, consider accuracy or F1-score.

    mse_best = mean_squared_error(y_test_outer, y_pred_best)
    mae_best = mean_absolute_error(y_test_outer, y_pred_best)
    rmse_best = np.sqrt(mse_best)
    r2_best = r2_score(y_test_outer, y_pred_best)
    time_duration = end_time - start_time

    mse_worst = mean_squared_error(y_test_outer, y_pred_worst)
    mae_worst = mean_absolute_error(y_test_outer, y_pred_worst)
    rmse_worst = np.sqrt(mse_worst)
    r2_worst = r2_score(y_test_outer, y_pred_worst)

    nested_scores.append({
        "Best Model": {
            "MSE": mse_best,
            "R^2": r2_best,
            "RMSE": rmse_best,
            "MAE": mae_best,
            "testing time": time_duration
        },
        "Worst Model": {
            "MSE": mse_worst,
            "R^2": r2_worst,
            "RMSE": rmse_worst,
            "MAE": mae_worst,
            "testing time": time_duration
        }
    })

# Save the best and worst hyperparameters
print("Best Hyperparameters:", best_model_hyperparameters)
print("Best MAE:", best_model_mae)

print("Worst Hyperparameters:", worst_model_hyperparameters)
print("Worst MAE:", worst_model_mae)

# Print or use the nested_scores as needed
print("Nested Cross-Validation Scores:")
for i, scores in enumerate(nested_scores):
    print(f"\nFold {i + 1} - MSE: {scores['Best Model']['MSE']}, R^2: {scores['Best Model']['R^2']}, RMSE: {scores['Best Model']['RMSE']}, MAE: {scores['Best Model']['MAE']}")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
4                 |4                 |num_transformer_blocks
232               |232               |head_size
3                 |3                 |num_heads
15                |15                |ff_dim
0.1204            |0.1204            |dropout_0
0.25362           |0.25362           |dropout_1
3                 |3                 |num_mlp_layers
128               |128               |mlp_units_0
0.19175           |0.19175           |mlp_dropout_0
adam              |adam              |optimizer
0.0012341         |0.0012341         |learning_rate

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.8256 - loss: 0.5739 - val_accuracy: 0.9281 - val_loss: 0.2083
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.9362 - loss: 0.2110 - val_accuracy: 0.9267 - val_loss: 0.2101
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9278 - loss: 0.2310 - val_accuracy: 0.9336 - v